This script is to calculate the average LCVC stat for every 49sqmi hex, the average LCVC stat for every subtype, and the average LCM stat for every subtype.

In [3]:
# Check Current Working Directory
import os
print(os.getcwd())

S:\Projects\ABC\y2025\Pro\Draft\ABC_RSR_Calculation_Pro\ABC_RSR_Calculation


In [2]:
# Change working directory to the notebook's directory if needed 
# This ensures the config.py file in the same directory as the notebook can be referenced
os.chdir(r'S:\Projects\ABC\y2025\Pro\Draft\ABC_RSR_Calculation_Pro\ABC_RSR_Calculation')

In [4]:
# Module search path: Also ensures that the directory containing 'config.py' is in the Python search path ('sys.path').
import sys
sys.path.append(r'S:\Projects\ABC\y2025\Pro\Draft\ABC_RSR_Calculation_Pro\ABC_RSR_Calculation')

In [5]:
import arcpy
import os
import config

Environments set from config file.
Paths set from config file.
File and variable names set from config file.


In [6]:
workspace_final = os.path.join(config.project_folder_final, f"{config.gdb_final}.gdb")
workspace_int = os.path.join(config.project_folder_int, f"{config.gdb_int}.gdb")

In [7]:
arcpy.env.workspace = workspace_int

In [12]:
# Calculate the average LCVC score for each 49sqmi hexagon.

lcvc_lookup = config.LCVC_Lookup
in_zone_data = config.wheel_grid
zone_field = "wheel_id"
in_value_raster = config.LCVC_Lookup
out_table = os.path.join(config.project_folder_int, config.gdb_int + ".gdb", config.LCVC_table)
ignore_nodata = "DATA"
statistics_type = "MEAN"
process_as_multidimensional = "CURRENT_SLICE"
percentile_values = 90
percentile_interpolation_type = "AUTO_DETECT"
circular_calculation = "ARITHMETIC"
circular_wrap_value = 360
out_join_layer = None

with arcpy.EnvManager(extent="MAXOF", cellSize = lcvc_lookup):
    with arcpy.EnvManager(extent="MAXOF", cellSize = lcvc_lookup):
        arcpy.sa.ZonalStatisticsAsTable(
            in_zone_data,
            zone_field,
            in_value_raster,
            out_table,
            ignore_nodata,
            statistics_type,
            process_as_multidimensional,
            percentile_values,
            percentile_interpolation_type,
            circular_calculation,
            circular_wrap_value,
            out_join_layer
        )
        
print("Table complete.")

Table complete.


In [8]:
# Calculate the average LCVC score for each subtype.

lcvc_lookup = config.LCVC_Lookup
in_zone_data = config.HabMap_tif
zone_field = "HabitatCod"
in_value_raster = config.LCVC_Lookup
out_table = os.path.join(config.project_folder_int, config.gdb_int + ".gdb", config.LCVC_subtype_table)
ignore_nodata = "DATA"
statistics_type = "MEAN"
process_as_multidimensional = "CURRENT_SLICE"
percentile_values = 90
percentile_interpolation_type = "AUTO_DETECT"
circular_calculation = "ARITHMETIC"
circular_wrap_value = 360
out_join_layer = None

with arcpy.EnvManager(extent="MAXOF", cellSize = lcvc_lookup):
    with arcpy.EnvManager(extent="MAXOF", cellSize = lcvc_lookup):
        arcpy.sa.ZonalStatisticsAsTable(
            in_zone_data,
            zone_field,
            in_value_raster,
            out_table,
            ignore_nodata,
            statistics_type,
            process_as_multidimensional,
            percentile_values,
            percentile_interpolation_type,
            circular_calculation,
            circular_wrap_value,
            out_join_layer
        )
        
print("Table complete.")

Table complete.


In [9]:
# Populate the final analysis table with the LCVC subtype mean scores.

# Define table names and join field
final_table = config.ABC_Analysis_Table
intermediate_table = config.LCVC_subtype_table
join_field = "HabitatCod"

# Step 1: Add a permanent field 'Conversion_Mean' in the final analysis table before applying the join
conversion_field = "Conversion_Mean"
arcpy.management.AddField(final_table, conversion_field, "DOUBLE")

# Step 2: Add the join
arcpy.management.AddJoin(final_table, join_field, intermediate_table, join_field)

# Step 3: Read values from the 'MEAN' field in the intermediate table
fields = ["{}.MEAN".format(intermediate_table), "{}.{}".format(final_table, conversion_field)]

# Use a search cursor to gather the MEAN values from the intermediate table
mean_values = []
with arcpy.da.SearchCursor(final_table, fields) as cursor:
    for row in cursor:
        mean_values.append(row[0])  # Store the 'MEAN' values

# Step 4: Remove the join before updating the table
arcpy.management.RemoveJoin(final_table)

# Step 5: Use an update cursor to populate the 'Conversion_Mean' field with the stored MEAN values
with arcpy.da.UpdateCursor(final_table, [conversion_field]) as cursor:
    for i, row in enumerate(cursor):
        row[0] = mean_values[i]  # Assign the stored 'MEAN' value to the 'Conversion_Mean' field
        cursor.updateRow(row)

print("Table updated.")

Table updated.


In [10]:
# Calculate the average LCM score for each subtype.

lcm_data = config.LCM_data
in_zone_data = config.HabMap_tif
zone_field = "HabitatCod"
in_value_raster = config.LCM_data
out_table = os.path.join(config.project_folder_int, config.gdb_int + ".gdb", config.LCM_subtype_table)
ignore_nodata = "DATA"
statistics_type = "MEAN"
process_as_multidimensional = "CURRENT_SLICE"
percentile_values = 90
percentile_interpolation_type = "AUTO_DETECT"
circular_calculation = "ARITHMETIC"
circular_wrap_value = 360
out_join_layer = None

with arcpy.EnvManager(extent="MAXOF", cellSize = lcm_data):
    with arcpy.EnvManager(extent="MAXOF", cellSize = lcm_data):
        arcpy.sa.ZonalStatisticsAsTable(
            in_zone_data,
            zone_field,
            in_value_raster,
            out_table,
            ignore_nodata,
            statistics_type,
            process_as_multidimensional,
            percentile_values,
            percentile_interpolation_type,
            circular_calculation,
            circular_wrap_value,
            out_join_layer
        )
        
print("Table complete.")

Table complete.


In [11]:
# Populate the final analysis table with the LCM subtype mean scores.

# Define table names and join field
final_table = config.ABC_Analysis_Table
intermediate_table = config.LCM_subtype_table
join_field = "HabitatCod"

# Step 1: Add a permanent field 'Condition_Mean' in the final analysis table before applying the join
conversion_field = "Condition_Mean"
arcpy.management.AddField(final_table, conversion_field, "DOUBLE")

# Step 2: Add the join
arcpy.management.AddJoin(final_table, join_field, intermediate_table, join_field)

# Step 3: Read values from the 'MEAN' field in the intermediate table
fields = ["{}.MEAN".format(intermediate_table), "{}.{}".format(final_table, conversion_field)]

# Use a search cursor to gather the MEAN values from the intermediate table
mean_values = []
with arcpy.da.SearchCursor(final_table, fields) as cursor:
    for row in cursor:
        mean_values.append(row[0])  # Store the 'MEAN' values

# Step 4: Remove the join before updating the table
arcpy.management.RemoveJoin(final_table)

# Step 5: Use an update cursor to populate the 'Conversion_Mean' field with the stored MEAN values
with arcpy.da.UpdateCursor(final_table, [conversion_field]) as cursor:
    for i, row in enumerate(cursor):
        row[0] = mean_values[i]  # Assign the stored 'MEAN' value to the 'Conversion_Mean' field
        cursor.updateRow(row)

print("Table updated.")

Table updated.


In [8]:
# Calculate the average Climate Velocity score for each subtype.

cv_data = config.CV_data
in_zone_data = config.HabMap_tif
zone_field = "HabitatCod"
in_value_raster = config.CV_data
out_table = os.path.join(config.project_folder_int, config.gdb_int + ".gdb", config.CV_subtype_table)
ignore_nodata = "DATA"
statistics_type = "MEAN"
process_as_multidimensional = "CURRENT_SLICE"
percentile_values = 90
percentile_interpolation_type = "AUTO_DETECT"
circular_calculation = "ARITHMETIC"
circular_wrap_value = 360
out_join_layer = None

with arcpy.EnvManager(extent="MAXOF", cellSize = cv_data):
    with arcpy.EnvManager(extent="MAXOF", cellSize = cv_data):
        arcpy.sa.ZonalStatisticsAsTable(
            in_zone_data,
            zone_field,
            in_value_raster,
            out_table,
            ignore_nodata,
            statistics_type,
            process_as_multidimensional,
            percentile_values,
            percentile_interpolation_type,
            circular_calculation,
            circular_wrap_value,
            out_join_layer
        )
        
print("Table complete.")

Table complete.


In [9]:
# Populate the final analysis table with the Climate Velocity subtype mean scores.

# Define table names and join field
final_table = config.ABC_Analysis_Table
intermediate_table = config.CV_subtype_table
join_field = "HabitatCod"

# Step 1: Add a permanent field 'Condition_Mean' in the final analysis table before applying the join
conversion_field = "Climate_Mean"
arcpy.management.AddField(final_table, conversion_field, "DOUBLE")

# Step 2: Add the join
arcpy.management.AddJoin(final_table, join_field, intermediate_table, join_field)

# Step 3: Read values from the 'MEAN' field in the intermediate table
fields = ["{}.MEAN".format(intermediate_table), "{}.{}".format(final_table, conversion_field)]

# Use a search cursor to gather the MEAN values from the intermediate table
mean_values = []
with arcpy.da.SearchCursor(final_table, fields) as cursor:
    for row in cursor:
        mean_values.append(row[0])  # Store the 'MEAN' values

# Step 4: Remove the join before updating the table
arcpy.management.RemoveJoin(final_table)

# Step 5: Use an update cursor to populate the 'Conversion_Mean' field with the stored MEAN values
with arcpy.da.UpdateCursor(final_table, [conversion_field]) as cursor:
    for i, row in enumerate(cursor):
        row[0] = mean_values[i]  # Assign the stored 'MEAN' value to the 'Climate_Mean' field
        cursor.updateRow(row)

print("Table updated.")

Table updated.


NOT NEEDED NOW: PULL IN ZONAL STATS FROM CLIMATE VELOCITY DATA

In [8]:
# Populate the final analysis table with the Climate Velocity mean scores.

# Define table names and join field
final_table = config.ABC_Analysis_Table
intermediate_table = "ABC35_ForwardVelocity"
join_field = "HabitatCod"

# Step 1: Add a permanent field 'Climate_Mean' in the final analysis table before applying the join
conversion_field = "Climate_Mean"
arcpy.management.AddField(final_table, conversion_field, "DOUBLE")

# Step 2: Read values from the intermediate table into a dictionary
mean_dict = {}
with arcpy.da.SearchCursor(intermediate_table, [join_field, "MEAN"]) as cursor:
    for row in cursor:
        habitat_code = row[0]
        mean_value = row[1]
        mean_dict[habitat_code] = mean_value  # Store HabitatCod as key and MEAN as value

# Step 3: Use an update cursor to populate the 'Climate_Mean' field with the corresponding MEAN values
with arcpy.da.UpdateCursor(final_table, [join_field, conversion_field]) as cursor:
    for row in cursor:
        habitat_code = row[0]
        row[1] = mean_dict.get(habitat_code, 0)  # Assign MEAN or 0 if no match
        cursor.updateRow(row)

print("Table updated.")

Table updated.
